In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Nullable{RigidBodyDynamics.SpatialInertia{Float64}}())

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
body2 = RigidBody(rand(SpatialInertia{Float64}, joint2.frameAfter));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
attach!(mechanism, body1, joint2, joint2ToBody1, body2);

In [5]:
x = MechanismState{Float64}(mechanism);
zero_configuration!(x)

In [6]:
num_positions(x)

8

In [7]:
num_velocities(x)

7

In [8]:
cache = FrameCache(mechanism, x);

In [9]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),0.08922778575134872 + 0.8566313487461696im - 0.11445055648658847jm + 0.4950981768448959km,FixedSizeArrays.Vec{3,Float64}((0.011584784675846338,0.8137088545185072,0.759491587195503)))

In [10]:
center_of_mass(mechanism, cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),FixedSizeArrays.Vec{3,Float64}((0.7173353954344781,1.371423344280339,1.4958987931110455)))

In [11]:
mass(mechanism)

1.3677585242785442

In [12]:
sum([isnull(body.inertia) ? 0. : get(body.inertia).mass for body in bodies(mechanism)])

1.3677585242785442

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root(mechanism).frame)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),FixedSizeArrays.Vec{3,Float64}((0.6377649248649085,0.8644042648111403,1.6518975062422028)))